In [20]:
# Basic libs
import os
import re
import numpy as np
import pandas as pd

# DATA PREPARATION

The following cells are left for reproducibility.
In a different machine, it is not necessary to run these cells.
These are only for condensing the data into a .csv file.

In [3]:
material_properties = pd.read_csv('thermal_properties.csv', index_col='material')
material_properties.head()

,Z,symbol,rho,k,cp
material,,,,,
aluminum,13,al,2698.0,225.940,921
americium*,95,am,11700.0,12.552,126
antimony,51,sb,6696.0,18.410,207
arsenic_grey,33,as,5730.0,4.184,331
barium,56,ba,3500.0,33.472,188


In [4]:
q_listA = np.arange(1.e7, 1.e8, 2e6)  # W/m3
q_listB = np.arange(1.e8, 5.e8, 2e7)  # W/m3
q_list = np.concatenate([q_listA, q_listB])
tau_list = [20, 40, 60, 80, 100]

In [13]:
ofile_location = '/home/roberto/Documents/phd-results/cfd/decayMoose/'

In [ ]:
# I want my .csv to look like this:
# index, material number, material, rho, k, cp, qvalue, q0, tau0

# another csv will have the time information
# index, time

In [29]:
data = []
for m_index, material in enumerate(material_properties[:33].index):
    for q_index, q0 in enumerate(q_list):
        for tau_index, tau in enumerate(tau_list):
            q_id = q_index * len(tau_list) + tau_index
            rho = material_properties.loc[material]['rho']
            k = material_properties.loc[material]['k']
            cp = material_properties.loc[material]['cp']

            ofile = f'input_m{m_index}_q{q_superindex}.csv'
            if ofile not in os.listdir(ofile_location):
                print(f'Discarded sample. {ofile}: does not exist.')
                continue

            full_path = os.path.join(ofile_location, ofile)
            df = pd.read_csv(full_path, usecols=['time', 'max_temp'])
            df.rename(columns={"time": "t", "max_temp": "T"}, inplace=True)

            temp = df[['T']].values.T[0]
            time = df[['t']].values.reshape(1, -1)[0]
            if m_index == 0 and q_id == 0:
                time0 = time.copy()

            if time[-1] != 500.0:
                print(f'Discarded sample. {ofile}: Run not completed.')
                continue

            if len(time) != len(time0):
                n_temp = np.interp(time0, time, temp)
                temp = n_temp.copy()

            # material number, material, rho, k, cp, qvalue, q0, tau0
            data.append([m_index, material, rho, k, cp, q_id, q0, tau] + temp.tolist())

Discarded sample. input_m7_q322.csv: Run not completed.
Discarded sample. input_m8_q323.csv: Run not completed.
Discarded sample. input_m10_q319.csv: Run not completed.
Discarded sample. input_m21_q312.csv: Run not completed.


In [34]:
# material number, material, rho, k, cp, qvalue, q0, tau0
columns = [
    'materialNumber',
    'material',
    'rho',
    'k',
    'cp',
    'q_id',
    'q0',
    'tau']

columns += [f'T{i}' for i in range(len(time0))]
columns

['materialNumber',
 'material',
 'rho',
 'k',
 'cp',
 'q_id',
 'q0',
 'tau',
 'T0',
 'T1',
 'T2',
 'T3',
 'T4',
 'T5',
 'T6',
 'T7',
 'T8',
 'T9',
 'T10',
 'T11',
 'T12',
 'T13',
 'T14',
 'T15',
 'T16',
 'T17',
 'T18',
 'T19',
 'T20',
 'T21',
 'T22',
 'T23',
 'T24',
 'T25',
 'T26',
 'T27',
 'T28',
 'T29',
 'T30',
 'T31',
 'T32',
 'T33',
 'T34',
 'T35',
 'T36',
 'T37',
 'T38',
 'T39',
 'T40',
 'T41',
 'T42',
 'T43',
 'T44',
 'T45',
 'T46',
 'T47',
 'T48',
 'T49',
 'T50']

In [43]:
df_data = pd.DataFrame(data, columns=columns)
df_data.head()

,materialNumber,material,rho,k,cp,q_id,q0,tau,T0,T1,...,T41,T42,T43,T44,T45,T46,T47,T48,T49,T50
0,0,aluminum,2698.0,225.94,921,0,10000000.0,20,25.020121,25.044322,...,25.859701,25.395992,25.160721,25.057753,25.018837,25.005917,25.001961,25.000733,25.000301,25.000176
1,0,aluminum,2698.0,225.94,921,1,10000000.0,40,25.020126,25.044341,...,29.235889,27.622515,26.468948,25.732850,25.320034,25.120172,25.038248,25.010311,25.002442,25.000931
2,0,aluminum,2698.0,225.94,921,2,10000000.0,60,25.020128,25.044347,...,33.160295,30.857515,28.916887,27.411235,26.346793,25.670735,25.291868,25.108430,25.033556,25.015102
3,0,aluminum,2698.0,225.94,921,3,10000000.0,80,25.020129,25.044350,...,36.534787,33.967522,31.603345,29.564059,27.927544,26.718577,25.907925,25.423127,25.169929,25.093661
4,0,aluminum,2698.0,225.94,921,4,10000000.0,100,25.020129,25.044352,...,39.269606,36.654246,34.108892,31.764257,29.728876,28.077059,26.837994,25.991209,25.473160,25.295364


In [44]:
df_time = pd.DataFrame(time0, columns=['time'])
df_time.head()

,time
0,0.010000
1,0.022000
2,0.036400
3,0.053680
4,0.074416


In [45]:
# Export to .csv
df_data.to_csv('data.csv')
df_time.to_csv('time.csv')